In [1]:
import pandas as pd
import string
import numpy as np
import glob
from keras.preprocessing.sequence import pad_sequences
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from tensorflow import keras

from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow as tf
from transformers import AutoTokenizer, DistilBertModel
from transformers import AdamW
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification


In [2]:

porter=PorterStemmer()
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']
import pickle
with open('tokenizer-200.pickle', 'rb') as handle:
    words = pickle.load(handle)

In [3]:
import json

words_list=json.loads(words.to_json())

In [4]:
y=json.loads(words_list["config"]["word_counts"])

In [5]:
words=list(y.keys())

In [6]:
words

['apron',
 'show',
 'plu',
 'fade',
 'quickli',
 'expos',
 'big',
 'deal',
 'within',
 'week',
 'use',
 'around',
 'fabric',
 'stitch',
 'attach',
 'one',
 'pocket',
 'toward',
 'center',
 'start',
 'rip',
 'not',
 'stay',
 'tear',
 'loos',
 'appear',
 'snag',
 'someth',
 'move',
 'shop',
 'expect',
 'otherwis',
 'ok',
 'price',
 'no',
 'dock',
 'web',
 'cam',
 'featur',
 'ever',
 'buy',
 'thisth',
 'zoom',
 'stuck',
 'never',
 'set',
 'foot',
 'danc',
 'studio',
 'might',
 'enjoy',
 'video',
 'anyon',
 '2',
 'lesson',
 'would',
 'feel',
 'wast',
 'money',
 'among',
 'us',
 'hasnt',
 'notic',
 'centuri',
 'human',
 'seem',
 'becom',
 'insul',
 'selfinvolv',
 'less',
 'mind',
 'other',
 'hope',
 'lynn',
 'truss',
 'offer',
 'humor',
 'observ',
 'subject',
 'along',
 'tongueincheek',
 'way',
 'possibl',
 'bit',
 'insight',
 'thought',
 'thing',
 'lot',
 'whine',
 'person',
 'petti',
 'annoy',
 'credit',
 'write',
 'style',
 'extrem',
 'witti',
 'find',
 'laugh',
 'loud',
 'read',
 'passa

In [7]:
def clean_punct(a):        
    return a.lower().translate(str.maketrans('', '', string.punctuation))

def clean_stop(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
        if tk in stopwords:
            continue
        else:
            str1 += tk+" "
            
    return str1

def clean_stem(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
            str1 += porter.stem(tk)+" "
            
    return str1

def clean_words(a):
    a_token=a.split()
    str1=""
    for tk in a_token:
        if tk in words:
            str1 += tk+" "
            
    return str1

In [8]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [9]:
def predict_proba(text_list, model, tokenizer):
  """
  To get array with predicted probabilities for 0 - instructions, 1- ingredients classes 
  for each paragraph in the list of strings
  :param text_list: list[str]
  :param model: transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification
  :param tokenizer: transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer
  :return res: numpy.ndarray
  """
     
  encodings = tokenizer(text_list, max_length=128, truncation=True, padding=True)
  dataset = tf.data.Dataset.from_tensor_slices((dict(encodings))) 
  preds = model.predict(dataset.batch(1)).logits
  res = tf.nn.softmax(preds, axis=1).numpy()
    
  return res

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.load_weights('./Modele_Bert/checkpoints2/my_checkpoint')

In [16]:
from tensorflow import keras



count=0;

pozitive_total=0
negative_total=0
pozitive_corecte=0
negative_corecte=0
neutre_total=0
neutre_corecte=0
txtfiles = []
for file in glob.glob("Stiri_pozitive/*.txt"):
    txtfiles.append(file)
    

print("---Stiri Pozitive---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        paragrafe=list(map(clean_stem, paragrafe))       
        paragrafe=list(map(clean_words, paragrafe))  
        
        results=predict_proba(paragrafe, model, tokenizer).tolist() 
       
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())

        procent_neg=sum_neg/nr_paragrafe-0.05
        procent_poz=sum_poz/nr_paragrafe+0.05

        print(str(file)+"  "+str(procent_neg)+"  "+str(procent_poz))
        
        pozitive_total+=1
        if procent_poz > 0.5 :
            pozitive_corecte+=1

     
print("Total stiri pozitive : "+str(pozitive_total))
print("Corecte stiri pozitive : "+str(pozitive_corecte)) 
print("Acuratete Pozitive: "+str(pozitive_corecte/pozitive_total))
print("")    

txtfiles = []
for file in glob.glob("Stiri_negative/*.txt"):
    txtfiles.append(file)
print("---Stiri Negative---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        paragrafe=list(map(clean_stem, paragrafe))       
        paragrafe=list(map(clean_words, paragrafe)) 
        
        results=predict_proba(paragrafe, model, tokenizer).tolist()  

    
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())

        
        procent_neg=sum_neg/nr_paragrafe-0.05
        procent_poz=sum_poz/nr_paragrafe+0.05

        print(str(file)+"  "+str(procent_neg)+"  "+str(procent_poz))
        
        negative_total+=1
        if procent_poz < 0.5:
            negative_corecte+=1
    
print("Total stiri negative : "+str(negative_total))
print("Corecte stiri negative : "+str(negative_corecte)) 
print("Acuratete negative: "+str(negative_corecte/negative_total))

print("")
print("Total stiri :" + str(pozitive_total+negative_total+neutre_total))
print("Total corecte :" + str(pozitive_corecte+negative_corecte+neutre_corecte))
print("Acuratete :" + str((pozitive_corecte+negative_corecte+neutre_corecte)/(pozitive_total+negative_total+neutre_total)))
    
    

---Stiri Pozitive---
Stiri_pozitive/34.txt  0.2721865024759213  0.7278135000138323
Stiri_pozitive/42.txt  0.543434668317576  0.4565653214693538
Stiri_pozitive/12.txt  0.6765518393066218  0.3234481553204686
Stiri_pozitive/41.txt  0.4231666070507705  0.576833390582118
Stiri_pozitive/11.txt  0.45329244358441495  0.5467075354902994
Stiri_pozitive/31.txt  0.393771314953962  0.6062286874986753
Stiri_pozitive/3.txt  0.29830853608027236  0.7016914689709405
Stiri_pozitive/39.txt  0.37389176963357096  0.626108223574673
Stiri_pozitive/17.txt  0.3613206084778843  0.6386793953776804
Stiri_pozitive/8.txt  0.2584538269120188  0.7415461794246271
Stiri_pozitive/10.txt  0.3751088792254694  0.6248911082395027
Stiri_pozitive/16.txt  0.5381797383613709  0.4618202504474547
Stiri_pozitive/23.txt  0.2574418468191134  0.7425581676873151
Stiri_pozitive/33.txt  0.12578100582415408  0.8742189865747486
Stiri_pozitive/30.txt  0.36618891695632627  0.63381107142484
Stiri_pozitive/18.txt  0.21299650609422072  0.787003

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.load_weights('./checkpoints/my_checkpoint')

In [ ]:
string1 = ["this is not good",
          "This is good",
          "This is shit",
          "This is the best decision of my life"]
paragrafe=list(map(clean_punct, string1))
paragrafe=list(map(clean_stop, string1))
paragrafe=list(map(clean_stem, string1))       
paragrafe=list(map(clean_words, string1)) 

predict_proba(string1, model, tokenizer).tolist()

In [ ]:
print(tf. __version__) 